In [2]:
import findspark
findspark.init()


In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[1]", "tweets_analysis")
ssc = StreamingContext(sc, 1)


22/07/17 20:46:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/17 20:46:35 WARN StreamingContext: spark.master should be set as local[n], n > 1 in local mode if you have receivers to get data, otherwise Spark jobs will not get resources to process the received data.


In [4]:
ssc.textFileStream("./tweets_json/")

In [5]:
strArray = "Hello World Hello Hadoop Hello spark kafka hive zookeeper hbase flume sqoop scala".split()
strArrayZipped = list(enumerate(strArray))
# print(list(map(lambda x: f"{x[0]} {x[1]}", strArrayZipped)))
strArrayTimestamped = list(map(lambda x: f"{x[0]} {x[1]}", strArrayZipped))
rdd = ssc.sparkContext.parallelize(strArrayTimestamped, len(strArrayTimestamped))
rdd.glom().collect()

[['0 Hello'],
 ['1 World'],
 ['2 Hello'],
 ['3 Hadoop'],
 ['4 Hello'],
 ['5 spark'],
 ['6 kafka'],
 ['7 hive'],
 ['8 zookeeper'],
 ['9 hbase'],
 ['10 flume'],
 ['11 sqoop'],
 ['12 scala']]

In [6]:
dStream = ssc.queueStream(rdds=[rdd])
counts = dStream.flatMap(lambda line: line.split(" "))\
                     .map(lambda word: (word, 1))\
                     .reduceByKey(lambda a, b: a+b)
counts.pprint(50)
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2022-07-17 20:46:49
-------------------------------------------
('0', 1)
('Hello', 3)
('1', 1)
('World', 1)
('2', 1)
('3', 1)
('Hadoop', 1)
('4', 1)
('5', 1)
('spark', 1)
('6', 1)
('kafka', 1)
('7', 1)
('hive', 1)
('8', 1)
('zookeeper', 1)
('9', 1)
('hbase', 1)
('10', 1)
('flume', 1)
('11', 1)
('sqoop', 1)
('12', 1)
('scala', 1)

-------------------------------------------
Time: 2022-07-17 20:46:50
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:46:51
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:46:52
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:46:53
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:46:54
-------------------------------------------

-------------------------------------------

KeyboardInterrupt: 

-------------------------------------------
Time: 2022-07-17 20:47:12
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:47:13
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:47:14
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:47:15
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:47:16
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:47:17
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:47:18
-------------------------------------------

-------------------------------------------
Time: 2022-07-17 20:47:19
-------------------------------------------



In [7]:
sc.stop()
ssc.stop()



In [ ]:
val strArray = "Hello World Hello Hadoop Hello spark kafka hive zookeeper hbase flume sqoop scala"
      .split("\\s+")
      .zipWithIndex
      .map{case (word,timestamp) => s"$timestamp $word"}
val rdd: RDD[String] = ssc.sparkContext.makeRDD(strArray)

    val wordCountStream: ConstantInputDStream[String] = new ConstantInputDStream(ssc,rdd)

    // 流式数据的处理
    wordCountStream.transform{
      rdd=>
      rdd.map{line => (line.split("\\s+")(1).toLowerCase,line)}  //考虑等值连接时的大小写问题
          .leftOuterJoin(blackListRDD)//不用等值连接，因为数据中不一定有黑名单中的词，用左外连接，因为把数据的单词都列出来
          .filter{case (_,(_,rightValue)) => !rightValue.getOrElse(false)}//左外连接的结果里，如果没有连接上就显示None, getOrElse(false)： 取出来就是true，取不处来的就是None，给它一个值false
          .map{case (_,(leftValue,_)) => leftValue}//把过滤的结果进行筛选，只需要（时间戳，单词）这样的数据形式
    }.print(20)